# [LAB-보충] 2. 시계열 데이터 전처리

## #01. 준비작업

### 1. 라이브러리 불러오기:

In [1]:
from hossam import load_data
from pandas import to_datetime
from pandas import DateOffset
from datetime import datetime as dt
from datetime import timedelta as td

### 2. 데이터 불러오기:

In [2]:
origin = load_data('shopping_orders')
origin.head()

[data] https://data.hossam.kr/data/lab03/shopping_orders.xlsx
[desc] 어느 쇼핑몰의 고객별 주문 내역 데이터
[!] Cannot read metadata


,고객명,주문일,배송완료일시,상품명,상품분류,주문금액,결제방법,배송지역
고객ID,,,,,,,,
C001,김민수,2024년 1월 15일,2024-01-17 14:30:00,무선 이어폰,전자기기,89000,카드결제,서울
C002,이영희,2024년 2월 3일,2024-02-05 16:20:00,겨울 코트,의류,156000,계좌이체,부산
C003,박철수,2024년 2월 14일,2024-02-16 11:45:00,발렌타인 초콜릿,식품,25000,카드결제,대구
C004,최지영,2024년 3월 8일,2024-03-10 13:15:00,여성 향수,화장품,78000,카드결제,인천
C005,정한솔,2024년 3월 22일,2024-03-24 09:30:00,게이밍 마우스,전자기기,65000,카드결제,서울


### 3. 데이터 타입 확인:

`주문일` 컬럼: 문자열, `배송완료일시` 컬럼: 날짜 타입

In [3]:
origin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, C001 to C018
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   고객명     30 non-null     object        
 1   주문일     30 non-null     object        
 2   배송완료일시  30 non-null     datetime64[ns]
 3   상품명     30 non-null     object        
 4   상품분류    30 non-null     object        
 5   주문금액    30 non-null     int64         
 6   결제방법    30 non-null     object        
 7   배송지역    30 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.1+ KB


## #02. 문자열을 날짜 타입으로 변환하기

주문일 column이 문자열이니까 날짜 계산을 위해서 날짜 타입으로 바꿔줘야 됨

In [4]:
df = origin.copy()

# 문자열에 대한 날짜 타입 변환:
# -> 년, 월, 일에 대한 format 지정 필요
df['주문일_타입변환'] = to_datetime(df['주문일'], format = '%Y년 %m월 %d일')

# 변환 전, 후의 데이터 타입 확인:
print(df[['주문일', '주문일_타입변환']].info())

# 변환 전, 후의 데이터 확인:
df[['주문일', '주문일_타입변환']].head()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, C001 to C018
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   주문일       30 non-null     object        
 1   주문일_타입변환  30 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 720.0+ bytes
None


,주문일,주문일_타입변환
고객ID,,
C001,2024년 1월 15일,2024-01-15
C002,2024년 2월 3일,2024-02-03
C003,2024년 2월 14일,2024-02-14
C004,2024년 3월 8일,2024-03-08
C005,2024년 3월 22일,2024-03-22


## #03. 날짜 타임 데이터에서 정보 추출

### 1. 년/월/일 정보 추출

날짜 타입 컬럼은 `.dt` 속성을 사용해서 다양한 정보를 추출할 수 있다.
년/월/일 별로 그룹화 해서 집계하는 것에도 쓰일 수 있음!

- .dt.year: 년도 추출
- .dt.month: 월 추출
- .dt.day: 일 추출

In [5]:
# 주문일에서 년/월/일 정보 추출:
df['주문년도'] = df['주문일_타입변환'].dt.year
df['주문월'] = df['주문일_타입변환'].dt.month
df['주문일자'] = df['주문일_타입변환'].dt.day

# 결과 확인:
df[['고객명', '주문일', '주문년도', '주문월', '주문일자']].head()

,고객명,주문일,주문년도,주문월,주문일자
고객ID,,,,,
C001,김민수,2024년 1월 15일,2024,1,15
C002,이영희,2024년 2월 3일,2024,2,3
C003,박철수,2024년 2월 14일,2024,2,14
C004,최지영,2024년 3월 8일,2024,3,8
C005,정한솔,2024년 3월 22일,2024,3,22


### 2. 요일 정보 추출:

In [6]:
# 주문일 기준 요일정보 추출:
df['주문요일영문'] = df['주문일_타입변환'].dt.day_name() # 영문 요일명
df['주문요일번호'] = df['주문일_타입변환'].dt.dayofweek # 0=월욜, 6=일욜

# 한글 요일명 매핑:
요일매핑 = {
    0: '월요일', 
    1: '화요일', 
    2: '수요일', 
    3: '목요일', 
    4: '금요일', 
    5: '토요일', 
    6: '일요일'
}
df['주문요일'] = df['주문요일번호'].map(요일매핑)

# 결과 확인:
df[['고객명', '주문일', '주문요일영문', '주문요일번호', '주문요일']].head()

,고객명,주문일,주문요일영문,주문요일번호,주문요일
고객ID,,,,,
C001,김민수,2024년 1월 15일,Monday,0,월요일
C002,이영희,2024년 2월 3일,Saturday,5,토요일
C003,박철수,2024년 2월 14일,Wednesday,2,수요일
C004,최지영,2024년 3월 8일,Friday,4,금요일
C005,정한솔,2024년 3월 22일,Friday,4,금요일


### 3. 분기와 몇 번쨰 날짜인지 추출:

In [7]:
df['주문분기'] = df['주문일_타입변환'].dt.quarter   # 분기(1~4)
df['연중몇일째'] = df['주문일_타입변환'].dt.day_of_year # 1년 중 몇 번째 날

# 결과 확인:
df[['고객명','주문일','주문분기','연중몇일째']].head()

,고객명,주문일,주문분기,연중몇일째
고객ID,,,,
C001,김민수,2024년 1월 15일,1,15
C002,이영희,2024년 2월 3일,1,34
C003,박철수,2024년 2월 14일,1,45
C004,최지영,2024년 3월 8일,1,68
C005,정한솔,2024년 3월 22일,1,82


### 4. 시/분/초 추출:

In [8]:
df['배송완료-시간'] = df['배송완료일시'].dt.hour
df['배송완료-분'] = df['배송완료일시'].dt.minute
df['배송완료-초'] = df['배송완료일시'].dt.second

# 결과 확인:
df[['고객명', '배송완료-시간', '배송완료-분', '배송완료-초']].head()

,고객명,배송완료-시간,배송완료-분,배송완료-초
고객ID,,,,
C001,김민수,14,30,0
C002,이영희,16,20,0
C003,박철수,11,45,0
C004,최지영,13,15,0
C005,정한솔,9,30,0


## #04. 날짜 계산하기

### 1. 날짜 간 차이 계산:

- 두 날짜간 차이: `(날짜2 - 날짜1).dt.days`
- 두 날짜 타입 간 연산은 timedelta 객체를 생성한 후
  - 이 객체에 대한 df 속성을 통해 두 날짜 간 차이를 구할 수 있다.

In [9]:
df['배송소요일'] = (df['배송완료일시'] - df['주문일_타입변환']).dt.days

# 결과 확인:
df[['주문일', '배송완료일시', '배송소요일']].head()

,주문일,배송완료일시,배송소요일
고객ID,,,
C001,2024년 1월 15일,2024-01-17 14:30:00,2
C002,2024년 2월 3일,2024-02-05 16:20:00,2
C003,2024년 2월 14일,2024-02-16 11:45:00,2
C004,2024년 3월 8일,2024-03-10 13:15:00,2
C005,2024년 3월 22일,2024-03-24 09:30:00,2


### 2. 경과일 구하기:

파이썬의 기본 내장 모듈인 datetime 객체를 사용해서 오늘 날짜를 생성하고,
오늘 날짜에서 DataFrame의 날짜 타입 컬럼을 빼면
해당 컬럼의 날짜와 오늘 날짜의 차이를 구할 수 있다.

In [10]:
오늘 = dt.now()
df['배송완료후_경과일'] = (오늘 - df['배송완료일시']).dt.days

# 결과 확인:
df[['주문일', '배송완료일시', '배송완료후_경과일']].head()

,주문일,배송완료일시,배송완료후_경과일
고객ID,,,
C001,2024년 1월 15일,2024-01-17 14:30:00,683
C002,2024년 2월 3일,2024-02-05 16:20:00,664
C003,2024년 2월 14일,2024-02-16 11:45:00,653
C004,2024년 3월 8일,2024-03-10 13:15:00,630
C005,2024년 3월 22일,2024-03-24 09:30:00,617


### 3. 기준일로부터 년/월/일 더하기, 빼기:

#### 1) 파이썬 내장 API 

- `datetime.Timedelta(days=n)`: n일 더하거나 빼기

In [11]:
# 재주문 예상일 (배송완료 후 30일)
df['재주문예상일'] = df['배송완료일시'] + td(days=30)

# 결과 확인:
df[['고객명', '배송완료일시', '재주문예상일']].head()

,고객명,배송완료일시,재주문예상일
고객ID,,,
C001,김민수,2024-01-17 14:30:00,2024-02-16 14:30:00
C002,이영희,2024-02-05 16:20:00,2024-03-06 16:20:00
C003,박철수,2024-02-16 11:45:00,2024-03-17 11:45:00
C004,최지영,2024-03-10 13:15:00,2024-04-09 13:15:00
C005,정한솔,2024-03-24 09:30:00,2024-04-23 09:30:00


#### 2) Pandas API 사용

- `pd.DateOffset(years=y, months=m, days=d, hours=h, minutes=i, seconds=s)`

In [12]:
# 1년 A/S 보장 종료일 (주문일 기준)
df['AS보장만료일'] = df['주문일_타입변환'] + DateOffset(years=1)

# 예상 매출 집계일 (주문일에서 3개월 후)
df['매출집계예정일'] = df['주문일_타입변환'] + DateOffset(months=3)

# 리뷰 요청 예정일시 (배송완료시점에서 7일 12시간 경과 후, 혼합사용 가능)
df['리뷰요청예정'] = df['배송완료일시'] + DateOffset(days=3, hours=12)

# 결과확인
df[['고객명', '주문일', 'AS보장만료일', '매출집계예정일', '리뷰요청예정']].head()

,고객명,주문일,AS보장만료일,매출집계예정일,리뷰요청예정
고객ID,,,,,
C001,김민수,2024년 1월 15일,2025-01-15,2024-04-15,2024-01-21 02:30:00
C002,이영희,2024년 2월 3일,2025-02-03,2024-05-03,2024-02-09 04:20:00
C003,박철수,2024년 2월 14일,2025-02-14,2024-05-14,2024-02-19 23:45:00
C004,최지영,2024년 3월 8일,2025-03-08,2024-06-08,2024-03-14 01:15:00
C005,정한솔,2024년 3월 22일,2025-03-22,2024-06-22,2024-03-27 21:30:00
